In [1]:
from datasets import load_dataset

data = load_dataset("squarelike/sharegpt_deepl_ko_translation")

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['num', 'korean', 'english'],
        num_rows: 200524
    })
})

In [3]:
type(data)

datasets.dataset_dict.DatasetDict

In [4]:
# 데이터 확인

print(data['train'][20000]['korean'])
print()
print(data['train'][20000]['english'])

행렬에 사용된 진리 값과 연결은 실제 개념에 기반하지 않기 때문에 이 특정 비결정적 행렬로 정확하게 모델링할 수 있는 실제 상황을 찾기는 어렵습니다.

일반적으로 비결정적 행렬 의미론은 불확실성 또는 여러 가지 가능한 결과가 있는 상황을 모델링하는 데 사용할 수 있습니다. 예를 들어 한 사람이 파티에 갈지 여부를 결정하는 상황을 생각해 보겠습니다. 이 사람이 취할 수 있는 행동에는 파티에 가거나 집에 머무르는 두 가지가 있습니다. 이 사람은 어떤 행동을 취해야 할지 불확실할 수 있으므로 "파티에 가자"라는 명제의 진리 값은 2(가능하지만 확실하지 않음을 나타냄)이고 "집에 머물러라"라는 명제의 진리 값은 3(가능하지만 확실하지 않음을 나타냄)일 수 있습니다.

그러나 이는 가능한 한 가지 예일 뿐이며 상황의 구체적인 세부 사항은 비결정적 행렬에 사용된 진리 값과 연결어의 특정 해석에 따라 달라질 수 있습니다.

도움이 되셨기를 바랍니다! 추가 질문이 있으시면 언제든지 알려주시기 바랍니다.

It is difficult to find a real-world situation that would be accurately modeled by this specific non-deterministic matrix, as the truth-values and connectives used in the matrix are not based on any real-world concepts.

In general, non-deterministic matrix semantics can be used to model situations where there is uncertainty or multiple possible outcomes. For example, consider a situation where a person is deciding whether or not to go to a party. There are two possible actions they can tak

In [5]:
# data 형태를 prompt에 맞추기
# korean과 english로 나뉜 데이터를 text로 통합
# datasets의 map함수 사용법 https://huggingface.co/docs/datasets/v2.14.4/en/process#map

def makedata(x):
    return {'text': f"### 영어: {x['english']}</끝>\n### 한국어: {x['korean']}</끝>"}

data = data.map(makedata)

In [6]:
data['train'][100]

{'num': 100,
 'korean': "회색조 배경과 컬러 피사체가 있는 이미지를 만들려면 사진 편집 소프트웨어를 사용하여 이미지의 색상을 선택적으로 조정할 수 있습니다.\n\n예를 들어 Adobe Photoshop에서는 '선택 및 마스크' 도구를 사용하여 사진의 피사체를 분리한 다음 나머지 이미지에 그레이 스케일 효과를 적용할 수 있습니다. 또는 '색조/채도' 조정 레이어를 사용하여 이미지의 색상을 선택적으로 조정하여 피사체는 컬러로 유지하면서 배경의 채도를 낮출 수 있습니다.\n\n다른 사진 편집 소프트웨어에도 이미지의 색상을 선택적으로 조정할 수 있는 유사한 도구 또는 옵션이 있을 수 있습니다. 이러한 도구를 사용하여 이미지에서 특정 색상 범위를 분리하고 채도 또는 색조를 조정하는 등 다른 색상 효과를 만들 수도 있습니다.",
 'english': 'If you want to create an image with a grayscale background and a colored subject, you can use photo editing software to selectively adjust the colors in the image.\n\nIn Adobe Photoshop, for example, you can use the "Select and Mask" tool to isolate the subject of the photograph and then apply a grayscale effect to the rest of the image. Alternatively, you can use the "Hue/Saturation" adjustment layer to selectively adjust the colors in the image, desaturating the background while leaving the subject in color.\n\nOther photo editing software may have similar tool

In [7]:
data['train'][150000]

{'num': 150000,
 'korean': '이 코드를 더 읽기 쉽게 만들 수 있는 몇 가지 방법이 있습니다:1. 읽기 쉽도록 여러 줄로 나눌 수 있습니다:```data-test={  dataTestAttribute + "-" +  col.label.toLowerCase().replace(" ", "").replace(".", "")}```1. 템플릿 문자열을 사용하여 가독성을 높이고 `+`로 문자열을 연결할 필요가 없도록 할 수도 있습니다:```data-test={`${데이터테스트속성}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`}```1. 마지막으로 코드를 더 간결하고 이해하기 쉽게 만들기 위해 도우미 함수를 정의할 수 있습니다:```함수 formatDataTestAttribute(dataTestAttribute, col) {  반환 `${데이터테스트어트리뷰트}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`;}// 다음과 같이 사용하세요:data-test={formatDataTestAttribute(dataTestAttribute, col)}```',
 'english': 'There are a few ways you could make this code more readable:\n\n1. You could split it into multiple lines to make it easier to read:\n```\ndata-test={\n  dataTestAttribute + "-" +\n  col.label.toLowerCase().replace(" ", "").replace(".", "")\n}\n```\n1. You could also use template strings to make it more readable and avoid having to concatenate strings with `+`:\n```\

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_id = "EleutherAI/polyglot-ko-5.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/200524 [00:00<?, ? examples/s]

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['num', 'korean', 'english', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 200524
    })
})

In [11]:
data["train"][150000]["text"]

'### 영어: There are a few ways you could make this code more readable:\n\n1. You could split it into multiple lines to make it easier to read:\n```\ndata-test={\n  dataTestAttribute + "-" +\n  col.label.toLowerCase().replace(" ", "").replace(".", "")\n}\n```\n1. You could also use template strings to make it more readable and avoid having to concatenate strings with `+`:\n```\ndata-test={`${dataTestAttribute}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`}\n```\n1. Finally, you could define a helper function to make the code more concise and easier to understand:\n```\nfunction formatDataTestAttribute(dataTestAttribute, col) {\n  return `${dataTestAttribute}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`;\n}\n\n// Then use it like this:\ndata-test={formatDataTestAttribute(dataTestAttribute, col)}\n```</끝>\n### 한국어: 이 코드를 더 읽기 쉽게 만들 수 있는 몇 가지 방법이 있습니다:1. 읽기 쉽도록 여러 줄로 나눌 수 있습니다:```data-test={  dataTestAttribute + "-" +  col.label.toLowerCase().replace(" ", ""

In [12]:
data["train"][150000]

{'num': 150000,
 'korean': '이 코드를 더 읽기 쉽게 만들 수 있는 몇 가지 방법이 있습니다:1. 읽기 쉽도록 여러 줄로 나눌 수 있습니다:```data-test={  dataTestAttribute + "-" +  col.label.toLowerCase().replace(" ", "").replace(".", "")}```1. 템플릿 문자열을 사용하여 가독성을 높이고 `+`로 문자열을 연결할 필요가 없도록 할 수도 있습니다:```data-test={`${데이터테스트속성}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`}```1. 마지막으로 코드를 더 간결하고 이해하기 쉽게 만들기 위해 도우미 함수를 정의할 수 있습니다:```함수 formatDataTestAttribute(dataTestAttribute, col) {  반환 `${데이터테스트어트리뷰트}-${col.label.toLowerCase().replace(" ", "").replace(".", "")}`;}// 다음과 같이 사용하세요:data-test={formatDataTestAttribute(dataTestAttribute, col)}```',
 'english': 'There are a few ways you could make this code more readable:\n\n1. You could split it into multiple lines to make it easier to read:\n```\ndata-test={\n  dataTestAttribute + "-" +\n  col.label.toLowerCase().replace(" ", "").replace(".", "")\n}\n```\n1. You could also use template strings to make it more readable and avoid having to concatenate strings with `+`:\n```\

In [13]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
def print_trainable_parameters(model):
    """
    Print the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainble%: {100 * trainable_params / all_param}"
    )

In [15]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


trainable params: 3670016 || all params: 3070156800 || trainble%: 0.11953838970048696


In [16]:
tokenNum_korean = 8611
tokenNum_english = 3029
tokenNum_colon = 29

In [17]:
import transformers
from transformers import Trainer
import numpy as np

class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def compute_loss(self, model, inputs, return_outputs=False):
    # print(inputs['labels'][1])
    for x in range(len(inputs['labels'])):
      if (inputs['labels'][x][3] == tokenNum_korean):
          maskindex = (inputs['labels'][x]==tokenNum_english).nonzero()[:, 0]
          temp = 0
          for i, index in enumerate(maskindex):
            if (inputs['labels'][x][index+1] != tokenNum_colon):
              maskindex = np.delete(maskindex.cpu(), i-temp)
              temp += 1

      elif (inputs['labels'][x][3] == tokenNum_english):
          maskindex = (inputs['labels'][x]==tokenNum_korean).nonzero()[:, 0]
          temp = 0
          for i, index in enumerate(maskindex):
            if (inputs['labels'][x][index+1] != tokenNum_colon):
              maskindex = np.delete(maskindex.cpu(), i-temp)
              temp += 1

      inputs['labels'][x][:maskindex[0]+2] = -100

    # print(inputs['labels'][1])
    outputs = model(**inputs)

    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [ ]:
# import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        fp16=True,
        output_dir="outputs",
        save_total_limit=2,
        logging_steps=300,
        report_to=["tensorboard"],
        num_train_epochs = 1,
        learning_rate=3e-4,
        resume_from_checkpoint=True,
        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train(resume_from_checkpoint=False)

/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
300,0.764100
600,0.661200
900,0.673100
1200,0.638900
1500,0.618600
1800,0.642500
2100,0.633700
2400,0.595000
2700,0.654600
3000,0.627900


In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.save_pretrained("./qrola/translation/5.8b/")

In [ ]:
# from transformers import StoppingCriteria, StoppingCriteriaList

# class StoppingCriteriaSub(StoppingCriteria):
#     def __init__(self, stops = [], encounters=1):
#         super().__init__()
#         self.stops = [stop for stop in stops]

#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
#         for stop in self.stops:
#             if torch.all((stop == input_ids[0][-len[stop]:])).item():
#                 return True
            
#         return False
    
# stop_words = ["</끝>"]
# stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
# stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = stop_words_ids)])


In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [ ]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=100,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [ ]:
gen(lan="ko", x="i am a student")

In [ ]:
print(model)